In [4]:
!pip install python-dotenv
!git init
!touch .gitignore
!echo "info.env" >> .gitignore
!git check-ignore -v .env


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [6]:
import pandas as pd
import http.client
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv("info.env")

# Access credentials
authUsername = os.getenv("API_USERNAME")
authPassword = os.getenv("API_PASSWORD")

#define URL for the API request
url = 'https://api.ravelry.com/color_families.json'
#make the request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
#close the connection
r.close()

print("response code: {}".format(r)) #tells me the response code
print("response code Details: {}".format(r.iter_lines())) #tells me details about the response code
print("response output:")
print(r.text) #tells me the output
print("response output formatted:")

query = 'bear'
page = 15
page_size = 1
craft = 'crochet'
#define URL for the API request
url = 'https://api.ravelry.com/patterns/search.json?query={}&page={}&page_size={}&craft={}'.format(query, page, page_size, craft)
#make the request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
#close the connection
r.close()
print("response code: {}\".format(r1)) #tells me the response code\n")
print("response code Details: {}\".format(r1.iter_lines())) #tells me details about the response code")
print("response output:")
print(r.text) #tells me the output
print("response output formatted:")
print(json.dumps(json.loads(r.text), indent=4)) #makes the json more readable

response code: <Response [200]>
response code Details: <generator object Response.iter_lines at 0x787503d0b740>
response output:
{"color_families": [{"color":null,"id":1,"name":"Yellow","permalink":"Yellow","spectrum_order":1},{"color":null,"id":2,"name":"Yellow-orange","permalink":"Yellow-orange","spectrum_order":2},{"color":null,"id":3,"name":"Orange","permalink":"Orange","spectrum_order":3},{"color":null,"id":4,"name":"Red-orange","permalink":"Red-orange","spectrum_order":4},{"color":null,"id":5,"name":"Red","permalink":"Red","spectrum_order":5},{"color":null,"id":6,"name":"Red-purple","permalink":"Red-purple","spectrum_order":0},{"color":null,"id":7,"name":"Purple","permalink":"Purple","spectrum_order":8},{"color":null,"id":8,"name":"Blue-purple","permalink":"Blue-purple","spectrum_order":9},{"color":null,"id":9,"name":"Blue","permalink":"Blue","spectrum_order":10},{"color":null,"id":10,"name":"Blue-green","permalink":"Blue-green","spectrum_order":11},{"color":null,"id":11,"name":"